In [90]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
from scipy import stats
import xarray as xr

In [91]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.7.1/'

In [92]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [93]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_SubsetC_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_SubsetC_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Begin_Time, Compl_Time, Peak_Time = [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [94]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [95]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [96]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [97]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [98]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [99]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/composites/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [100]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
Control_B_Compo_nc = Composite_File('composite_h3_CORI.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
Control_C_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')

In [101]:
# Open Storm Composite Files
RCP45_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP45_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
RCP45_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')

In [102]:
# Open Storm Composite Files
RCP85_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP85_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
RCP85_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004.nc')

In [103]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon)
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [104]:
# Combine Composite DFs
def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
    Compo_DF_A = Composite_DF(Compo_A, "A")
    Compo_DF_B = Composite_DF(Compo_B, "B")
    Compo_DF_C = Composite_DF(Compo_C, "C")
    Compo_DF = pandas.concat([Compo_DF_A, Compo_DF_B, Compo_DF_C]).reset_index()
    Compo_DF = Compo_DF.drop("index", axis=1)
    return (Compo_DF)

In [105]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [106]:
RCP45_Compo = Combine_Compo_DF(RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [107]:
RCP85_Compo = Combine_Compo_DF(RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [108]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [109]:
# Create Function to Find Year Diff
def Year_Diff_Find(New_Time):
    Years = [1900,1930,1960,2000,2031,2062,2100,2131,2162,2193]
    New_Time_Index = -728
    for i in range(len(Years)):
        if i < 3:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 1985 - Years[i]
                New_Time_Index = i
        elif i < 6:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
        else:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
    if New_Time_Index % 3 == 0:
        ABC = "A"
    elif New_Time_Index % 3 == 1:
        ABC = "B"
    elif New_Time_Index % 3 == 2:
        ABC = "C"
    return (int(Year_Diff), ABC)

In [110]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF):
    DF_Storm = Find_Storm(Data_DF, Code)
    Code_List = DF_Storm["Code"]
    Name_List = DF_Storm["Name"]
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    SLP = DF_Storm["SLP(hPa)"]
    Storm_Phase = DF_Storm["Storm Phase"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Year_Diff, ABC = Year_Diff_Find(New_Time[0])
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
# Find Possible Storms that Occur at the Same Time
        Compo_Storm = Compo_DF[(Compo_DF["ABC"] == ABC) & (Compo_DF["Time"] == Orig_Time)].reset_index()
# If No Storm Found:
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
# Storms Found:
        else:
            Dist_Min = [7428,-728]
            for c in range(len(Compo_Storm)):
                Dist = Find_Distance(Lat[i], Compo_Storm["Lat"][c], Lon[i], Compo_Storm["Lon"][c])
# Find Storm Closest to Storm Center
                if Dist < Dist_Min[0]:
# At Most 300km of Error in Location Permitted
                    if Dist < 300:
                        Dist_Min = [Dist, Compo_Storm["Orig Index"][c]]
                    else:
                        Dist_Min = [Dist, -728]
            Compo_Indexes[i] = Dist_Min[1]
    DF_Storm_Compo_Init = pandas.DataFrame({"Code": Code_List, "Name": Name_List, \
    "Compo Index": Compo_Indexes, "Time": New_Time, \
    "Lon": Lon, "Lat": Lat, "SLP": SLP, "Storm Phase": Storm_Phase})
# Remove Datapoints With Missing Compo Index
    DF_Storm_Compo = DF_Storm_Compo_Init[DF_Storm_Compo_Init["Compo Index"] >= 0].reset_index()
    DF_Storm_Compo = DF_Storm_Compo.drop("index", axis=1)
    return (DF_Storm_Compo)

In [111]:
def Windspeed_850hPa(Compo_nc, Compo_Index):
    U850 = numpy.array(Compo_nc.snap_U850[int(Compo_Index)])
    V850 = numpy.array(Compo_nc.snap_V850[int(Compo_Index)])
    Snap_850 = numpy.sqrt(U850 **2 + V850 **2)
    return (Snap_850)

In [112]:
# Find Precip Rate From Compo File
def Precip_Rate(Compo_nc, Compo_Index):
    Precip_ms = numpy.array(Compo_nc.snap_PRECT[int(Compo_Index)])
    Precip_mmhr = Precip_ms * 3600 * 1000
    return (Precip_mmhr)

In [113]:
# Find Precipitable Water From Compo File
def Precip_Water(Compo_nc, Compo_Index):
    Precipitable_Water = numpy.array(Compo_nc.snap_TMQ[int(Compo_Index)])
    return (Precipitable_Water)

In [114]:
# Find Outgoing Longwave Radiation and Cloud Top Temperature From Compo File
def Cloud_Temp(Compo_nc, Compo_Index):
    Outgoing_Longwave = numpy.array(Compo_nc.snap_FLUT[int(Compo_Index)])
    Sigma = 5.67 * 10**-8
    Cloud_Temp_K = (Outgoing_Longwave / (0.95 * Sigma)) ** 0.25
    Cloud_Temp_C = Cloud_Temp_K - 273.15
    return (Cloud_Temp_C)

In [115]:
# Find Surface Temperature From Compo File
def Temp_Surface(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_TS[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [116]:
# Find 850hPa Temperature From Compo File
def Temp_850hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T850[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [117]:
# Find 500hPa Temperature From Compo File
def Temp_500hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T500[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [118]:
# Find 200hPa Temperature From Compo File
def Temp_200hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T200[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [119]:
# Find 500hPa Vertical Velocity From Compo File
def Omega_500hPa(Compo_nc, Compo_Index):
    Vert_Velo = numpy.array(Compo_nc.snap_OMEGA500[int(Compo_Index)])
    return (Vert_Velo)

In [120]:
# Find 200hPa Zonal Winds From Compo File
def U_200hPa(Compo_nc, Compo_Index):
    Zonal_Wind = numpy.array(Compo_nc.snap_U200[int(Compo_Index)])
    return (Zonal_Wind)

In [121]:
# Find 850hPa Max Windspeed and Wind Field Size at Each 6 Hourly Data Point
def Wind_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
    SLP = DF_Storm_Compo["SLP"]
#
# Create Array to Store Data
    Wind_Field_Info = numpy.zeros((6,len(Compo_Index)))
    Wind_Field_Info[0] = SLP
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Windspeed Snap From Compo_nc
        Snap_850 = Windspeed_850hPa(Compo_nc, Compo_Index[k])
# Find Maximum 850hPa Windspeed
        Windspeed_850 = numpy.max(Snap_850)
        Wind_Field_Info[1][k] = Windspeed_850
# Count Number of Data Points With Windspeed Above 13,18,25,33m/s
        Snap_Sort = numpy.sort(Snap_850.ravel())
        Count_13 = len(Snap_Sort[Snap_Sort >= 13])
        Count_18 = len(Snap_Sort[Snap_Sort >= 18])
        Count_25 = len(Snap_Sort[Snap_Sort >= 25])
        Count_33 = len(Snap_Sort[Snap_Sort >= 33])
        Wind_Field_Info[2][k] = Count_13
        Wind_Field_Info[3][k] = Count_18
        Wind_Field_Info[4][k] = Count_25
        Wind_Field_Info[5][k] = Count_33
#
# Add Wind Field Info Into DF Storm Compo
    DF_Storm_Compo["850hPa Winds"] = Wind_Field_Info[1]
    DF_Storm_Compo["13m/s"] = Wind_Field_Info[2]
    DF_Storm_Compo["18m/s"] = Wind_Field_Info[3]
    DF_Storm_Compo["25m/s"] = Wind_Field_Info[4]
    DF_Storm_Compo["33m/s"] = Wind_Field_Info[5]
    return (DF_Storm_Compo)

In [122]:
# Find Max Precip and Total Precip Over Area
def Precip_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
#
# Create Array to Store Data
    Precip_Field_Info = numpy.zeros((7,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find Precip Snap From Compo_nc
        Snap_Precip = Precip_Rate(Compo_nc, Compo_Index[k])
# Find Maximum Precip Rate
        Max_Precip = numpy.max(Snap_Precip)
        Precip_Field_Info[0][k] = Max_Precip
# Find Total Areal Precip
        Snap_Sort = numpy.sort(Snap_Precip.ravel())
        Areal_Precip_Total = numpy.sum(Snap_Sort)
        Precip_Field_Info[1][k] = Areal_Precip_Total
# Count Number of Data Points With Precip Rate Above 1, 5, 10mm/hr
        Count_1 = len(Snap_Sort[Snap_Sort >= 1])
        Count_5 = len(Snap_Sort[Snap_Sort >= 5])
        Count_10 = len(Snap_Sort[Snap_Sort >= 10])
        Precip_Field_Info[2][k] = Count_1
        Precip_Field_Info[3][k] = Count_5
        Precip_Field_Info[4][k] = Count_10
#
# Find Precipitable Water Snap From Compo_nc
        Snap_Precip_Water = Precip_Water(Compo_nc, Compo_Index[k])
# Find Maximum Precipitable Water
        Max_Precip_Water = numpy.max(Snap_Precip_Water)
        Precip_Field_Info[5][k] = Max_Precip_Water
# Find Areal Precipitable Water Total
        Precip_Water_Total = numpy.sum(Snap_Precip_Water.ravel())
        Precip_Field_Info[6][k] = Precip_Water_Total
# 
# Add Precip Field Info Into DF Storm Compo
    DF_Storm_Compo["Max Precip Rate"] = Precip_Field_Info[0]
    DF_Storm_Compo["Total Areal Precip"] = Precip_Field_Info[1]
    DF_Storm_Compo["1mm/hr"] = Precip_Field_Info[2]
    DF_Storm_Compo["5mm/hr"] = Precip_Field_Info[3]
    DF_Storm_Compo["10mm/hr"] = Precip_Field_Info[4]
    DF_Storm_Compo["Max Precip Water"] = Precip_Field_Info[5]
    DF_Storm_Compo["Total Precip Water"] = Precip_Field_Info[6]
    return (DF_Storm_Compo)

In [123]:
# Find Temperature Variables
def Temp_Vars_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
#
# Create Array to Store Data
    Temp_Field_Info = numpy.zeros((8,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find Each Variable From Compo_nc
        Snap_Temp_Cloud = Cloud_Temp(Compo_nc, Compo_Index[k])
        Snap_Temp_Sfc = Temp_Surface(Compo_nc, Compo_Index[k])
        Snap_Temp_850hPa = Temp_850hPa(Compo_nc, Compo_Index[k])
        Snap_Temp_500hPa = Temp_500hPa(Compo_nc, Compo_Index[k])
        Snap_Temp_200hPa = Temp_200hPa(Compo_nc, Compo_Index[k])
        Snap_Omega_500hPa = Omega_500hPa(Compo_nc, Compo_Index[k])
        Snap_U_200hPa = U_200hPa(Compo_nc, Compo_Index[k])
#
# Find Minimum Cloud Top Temperature
        Min_Temp_Cloud = numpy.min(Snap_Temp_Cloud)
# Find Mean Surface, 850hPa, 500hPa, 200hPa Temperatures Within 1 Lat/Lon of Storm Center
        Mean_Temp_Sfc = numpy.mean(Snap_Temp_Sfc[17:23,17:23])
        Mean_Temp_850hPa = numpy.mean(Snap_Temp_850hPa[17:23,17:23])
        Mean_Temp_500hPa = numpy.mean(Snap_Temp_500hPa[17:23,17:23])
        Mean_Temp_200hPa = numpy.mean(Snap_Temp_200hPa[17:23,17:23])
        Mean_U_200hPa = numpy.mean(Snap_U_200hPa[17:23,17:23])
# Find Maximum Rising and Sinking Vertical Velocity
        Min_Omega = numpy.min(Snap_Omega_500hPa) * -1
        Max_Omega = numpy.max(Snap_Omega_500hPa)
#
# Add To Array
        Temp_Field_Info[0][k] = Min_Temp_Cloud
        Temp_Field_Info[1][k] = Mean_Temp_Sfc
        Temp_Field_Info[2][k] = Mean_Temp_850hPa
        Temp_Field_Info[3][k] = Mean_Temp_500hPa
        Temp_Field_Info[4][k] = Mean_Temp_200hPa
        Temp_Field_Info[5][k] = Mean_U_200hPa
        Temp_Field_Info[6][k] = Min_Omega
        Temp_Field_Info[7][k] = Max_Omega
#
# Add To DF Storm Compo
    DF_Storm_Compo["Min Cloud Temp"] = Temp_Field_Info[0]
    DF_Storm_Compo["Sfc Temp"] = Temp_Field_Info[1]
    DF_Storm_Compo["850hPa Temp"] = Temp_Field_Info[2]
    DF_Storm_Compo["500hPa Temp"] = Temp_Field_Info[3]
    DF_Storm_Compo["200hPa Temp"] = Temp_Field_Info[4]
    DF_Storm_Compo["200hPa U"] = Temp_Field_Info[5]
    DF_Storm_Compo["Max Rising"] = Temp_Field_Info[6]
    DF_Storm_Compo["Max Sinking"] = Temp_Field_Info[7]
    return (DF_Storm_Compo)

In [124]:
# Create New Data DF
def DF_Data_Compo(Data_DF, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
    ABC_List = ET_DF["ABC"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        DF_Storm_Compo = Find_Composite_Data(Code_List[n], Data_DF, Compo_DF)
# Find Which Compo nc To Use
        if ABC_List[n] == "A":
            Compo_nc = Compo_nc_A
        elif ABC_List[n] == "B":
            Compo_nc = Compo_nc_B
        elif ABC_List[n] == "C":
            Compo_nc = Compo_nc_C
# Apply Functions For Finding Wind Field and Precip Field
        DF_Storm_Compo = Wind_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Precip_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Temp_Vars_Find(DF_Storm_Compo, Compo_nc)
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0:
            if DF_Storm_Compo["Storm Phase"][len(DF_Storm_Compo)-1] == "Extratropical":
# Combine DF Storm Compos
                try:
                    Data_Compo = pandas.concat([Data_Compo, DF_Storm_Compo])
                except:
                    Data_Compo = DF_Storm_Compo.copy()
    Data_Compo_Final = Data_Compo.reset_index().drop("index", axis=1)
    return (Data_Compo_Final)

In [125]:
# Create New ET DF
def DF_ET_Compo(Data_Compo, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        ET_Storm = Find_Storm(ET_DF, Code_List[n])
        DF_Storm_Compo = Find_Storm(Data_Compo, Code_List[n])
# Find ET Begin and ET Complete Time
        Trop_Peak_Time = ET_Storm["Trop Peak Time"][0]
        Begin_Time = ET_Storm["ET Begin Time"][0]
        Compl_Time = ET_Storm["ET Complete Time"][0]
        DF_Trop_Peak = DF_Storm_Compo[DF_Storm_Compo["Time"] == Trop_Peak_Time].reset_index()
        DF_Begin = DF_Storm_Compo[DF_Storm_Compo["Time"] == Begin_Time].reset_index()
        DF_Compl = DF_Storm_Compo[DF_Storm_Compo["Time"] == Compl_Time].reset_index()
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0 and len(DF_Trop_Peak) and len(DF_Begin) > 0 and len(DF_Compl) > 0:
# Combine ET Storm Compos
            ET_Storm_Compo = Find_ET_Compo(Code_List[n], ET_Storm, DF_Trop_Peak, DF_Begin, DF_Compl)
            try:
                ET_Compo = pandas.concat([ET_Compo, ET_Storm_Compo])
            except:
                ET_Compo = ET_Storm_Compo.copy()
        else:
            print (Code_List[n], len(DF_Trop_Peak), len(DF_Begin), len(DF_Compl))
    ET_Compo_Final = ET_Compo.reset_index().drop("index", axis=1)
    return (ET_Compo_Final)

In [126]:
def Find_ET_Compo(Code, ET_Storm, DF_Trop_Peak, DF_Begin, DF_Compl):
    ET_Storm_Compo = ET_Storm[["Code", "Name", "Trop Peak Time", "ET Begin Time", "ET Complete Time", \
    "Trop Peak SLP", "ET Begin SLP", "ET Complete SLP"]].copy()
    Vars = ["850hPa Winds", "13m/s", "18m/s", "25m/s", "33m/s", \
    "1mm/hr", "5mm/hr", "10mm/hr", "Max Precip Rate", "Max Precip Water", "Total Precip Water", \
    "Min Cloud Temp", "Sfc Temp", "850hPa Temp", "500hPa Temp", "200hPa Temp", "200hPa U", \
    "Max Rising", "Max Sinking"]
    for m in range(len(Vars)):
        Var = Vars[m]
        Trop_Peak_Var = str("Trop Peak " + Var)
        Begin_Var = str("ET Begin " + Var)
        Compl_Var = str("ET Complete " + Var)
        ET_Storm_Compo[Trop_Peak_Var] = DF_Trop_Peak[Var][0]
        ET_Storm_Compo[Begin_Var] = DF_Begin[Var][0]
        ET_Storm_Compo[Compl_Var] = DF_Compl[Var][0]
    return (ET_Storm_Compo)

In [127]:
Control_Data_Compo = DF_Data_Compo(Control_Data, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [128]:
Control_ET_Compo = DF_ET_Compo(Control_Data_Compo, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [129]:
RCP45_Data_Compo = DF_Data_Compo(RCP45_Data, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [130]:
RCP45_ET_Compo = DF_ET_Compo(RCP45_Data_Compo, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [131]:
RCP85_Data_Compo = DF_Data_Compo(RCP85_Data, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [132]:
RCP85_ET_Compo = DF_ET_Compo(RCP85_Data_Compo, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [133]:
# Calculate 10%, Median, 90% Percentiles
def Percentile(Array):
    Percent_10 = round(numpy.nanpercentile(Array, 10), 3)
    Median = round(numpy.nanmedian(Array), 3)
    Percent_90 = round(numpy.nanpercentile(Array, 90), 3)
    return ([Percent_10, Median, Percent_90])

In [134]:
# Calculate Statistical Significance Using KS Test
def KS_Test(Control_Array, RCP45_Array, RCP85_Array):
    P_Val_RCP45 = round(stats.kstest(Control_Array, RCP45_Array)[1], 3)
    P_Val_RCP85 = round(stats.kstest(Control_Array, RCP85_Array)[1], 3)
    return (P_Val_RCP45, P_Val_RCP85)

In [135]:
# Create DataFrame to Store Percentiles Data
def Percentile_DF(Control_Array, RCP45_Array, RCP85_Array):
    Control_Percentiles = Percentile(Control_Array)
    RCP45_Percentiles = Percentile(RCP45_Array)
    RCP85_Percentiles = Percentile(RCP85_Array)
    P_Vals = KS_Test(Control_Array, RCP45_Array, RCP85_Array)
    Control_Percentiles.append(1.000)
    RCP45_Percentiles.append(P_Vals[0])
    RCP85_Percentiles.append(P_Vals[1])
    DF = pandas.DataFrame({"Percentile": ["10%", "Median", "90%", "P Val"], \
    "Control": Control_Percentiles, "RCP4.5": RCP45_Percentiles, "RCP8.5": RCP85_Percentiles})
    return (DF)

In [136]:
# Latitude
Var = "Trop Peak Lat"
Trop_Peak_Lat_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
Trop_Peak_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,20.400,24.677,23.530
1,Median,29.160,32.045,31.940
2,90%,37.682,39.081,38.206
3,P Val,NaN,0.101,0.049


In [137]:
# Latitude
Var = "ET Begin Lat"
ET_Begin_Lat_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
ET_Begin_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,26.338,31.378,31.960
1,Median,37.470,39.085,39.790
2,90%,45.896,47.263,45.840
3,P Val,NaN,0.190,0.147


In [138]:
# Latitude
Var = "ET Complete Lat"
ET_Compl_Lat_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
ET_Compl_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,35.984,41.778,41.248
1,Median,46.940,47.075,46.360
2,90%,50.378,51.041,50.582
3,P Val,NaN,0.401,0.672


In [139]:
# Longitude
Var = "Trop Peak Lon"
Trop_Peak_Lon_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
Trop_Peak_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-79.336,-79.347,-79.990
1,Median,-64.920,-59.460,-61.690
2,90%,-44.894,-47.184,-49.098
3,P Val,NaN,0.217,0.547


In [140]:
# Longitude
Var = "ET Begin Lon"
ET_Begin_Lon_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
ET_Begin_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-75.98,-74.068,-76.382
1,Median,-62.44,-54.420,-56.900
2,90%,-35.97,-36.337,-36.174
3,P Val,NaN,0.159,0.109


In [141]:
# Longitude
Var = "ET Complete Lon"
ET_Compl_Lon_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
ET_Compl_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-66.396,-63.825,-64.312
1,Median,-46.110,-42.580,-44.600
2,90%,-22.440,-22.793,-28.008
3,P Val,NaN,0.211,0.155


In [142]:
# Sea Level Pressure
Var = "Trop Peak SLP"
Trop_Peak_SLP_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
Trop_Peak_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,924.968,920.226,910.282
1,Median,954.470,949.540,949.320
2,90%,984.564,987.147,984.918
3,P Val,NaN,0.575,0.070


In [143]:
# Sea Level Pressure
Var = "ET Begin SLP"
ET_Begin_SLP_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
ET_Begin_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,939.880,937.625,927.014
1,Median,977.650,975.665,972.580
2,90%,999.674,997.698,1002.640
3,P Val,NaN,0.591,0.558


In [144]:
# Sea Level Pressure
Var = "ET Complete SLP"
ET_Compl_SLP_DF = Percentile_DF(Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
ET_Compl_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,971.716,972.812,970.790
1,Median,990.920,990.625,988.380
2,90%,1003.096,1002.951,1003.446
3,P Val,NaN,0.540,0.263


In [145]:
# 850hPa Winds
Var = "Trop Peak 850hPa Winds"
Trop_Peak_850hPa_Winds_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_850hPa_Winds_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,38.473,39.114,41.183
1,Median,56.550,57.944,59.479
2,90%,72.015,72.319,76.878
3,P Val,NaN,0.762,0.347


In [146]:
# 850hPa Winds
Var = "ET Begin 850hPa Winds"
ET_Begin_850hPa_Winds_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_850hPa_Winds_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,31.105,33.248,31.567
1,Median,48.065,51.208,50.174
2,90%,63.812,68.103,71.389
3,P Val,NaN,0.503,0.375


In [147]:
# 850hPa Winds
Var = "ET Complete 850hPa Winds"
ET_Compl_850hPa_Winds_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_850hPa_Winds_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,30.174,27.242,31.484
1,Median,38.289,39.343,40.114
2,90%,47.580,48.844,51.401
3,P Val,NaN,0.379,0.162


In [148]:
# 13m/s Wind Field
Var = "Trop Peak 13m/s"
Trop_Peak_Wind_13ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Wind_13ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,473.0,472.000,513.000
1,Median,945.0,1091.500,899.000
2,90%,2060.0,1936.800,2621.600
3,P Val,NaN,0.321,0.286


In [149]:
# 13m/s Wind Field
Var = "ET Begin 13m/s"
ET_Begin_Wind_13ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Wind_13ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,586.2,938.200,787.6
1,Median,1367.0,1528.000,1619.0
2,90%,2372.0,2938.400,2918.6
3,P Val,NaN,0.247,0.1


In [150]:
# 13m/s Wind Field
Var = "ET Complete 13m/s"
ET_Compl_Wind_13ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Wind_13ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,1142.8,1463.600,1525.200
1,Median,2225.0,2198.500,2455.000
2,90%,3358.4,3434.000,3366.200
3,P Val,NaN,0.888,0.295


In [151]:
# 18m/s Wind Field
Var = "Trop Peak 18m/s"
Trop_Peak_Wind_18ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Wind_18ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,154.6,216.400,198.400
1,Median,388.0,447.000,491.000
2,90%,711.2,854.600,1344.200
3,P Val,NaN,0.185,0.051


In [152]:
# 18m/s Wind Field
Var = "ET Begin 18m/s"
ET_Begin_Wind_18ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Wind_18ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,192.0,361.90,320.200
1,Median,565.0,692.50,711.000
2,90%,1150.4,1447.30,1501.400
3,P Val,NaN,0.07,0.116


In [153]:
# 18m/s Wind Field
Var = "ET Complete 18m/s"
ET_Compl_Wind_18ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Wind_18ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,400.0,416.700,555.800
1,Median,1005.0,966.500,1093.000
2,90%,1718.2,2084.200,2099.600
3,P Val,NaN,0.685,0.282


In [154]:
# 25m/s Wind Field
Var = "Trop Peak 25m/s"
Trop_Peak_Wind_25ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Wind_25ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,61.6,67.100,58.000
1,Median,142.0,169.000,215.000
2,90%,359.6,356.200,611.600
3,P Val,NaN,0.273,0.006


In [155]:
# 25m/s Wind Field
Var = "ET Begin 25m/s"
ET_Begin_Wind_25ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Wind_25ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,54.4,73.700,56.800
1,Median,187.0,237.500,271.000
2,90%,456.4,563.200,559.800
3,P Val,NaN,0.279,0.125


In [156]:
# 25m/s Wind Field
Var = "ET Complete 25m/s"
ET_Compl_Wind_25ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Wind_25ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,60.4,18.800,100.000
1,Median,259.0,301.500,303.000
2,90%,667.2,852.900,832.800
3,P Val,NaN,0.945,0.369


In [157]:
# 33m/s Wind Field
Var = "Trop Peak 33m/s"
Trop_Peak_Wind_33ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Wind_33ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,13.2,15.000,17.000
1,Median,59.0,71.000,93.000
2,90%,153.8,158.500,193.600
3,P Val,NaN,0.555,0.017


In [158]:
# 33m/s Wind Field
Var = "ET Begin 33m/s"
ET_Begin_Wind_33ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Wind_33ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,0.0,3.500,0.000
1,Median,55.0,82.000,94.000
2,90%,215.6,201.500,270.600
3,P Val,NaN,0.224,0.089


In [159]:
# 33m/s Wind Field
Var = "ET Complete 33m/s"
ET_Compl_Wind_33ms_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Wind_33ms_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,0.0,0.000,0.000
1,Median,40.0,48.000,67.000
2,90%,278.2,217.900,337.400
3,P Val,NaN,0.934,0.364


In [160]:
# 1mm/hr Precip Field
Var = "Trop Peak 1mm/hr"
Trop_Peak_Precip_1mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Precip_1mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,118.0,90.700,76.400
1,Median,227.0,207.000,260.000
2,90%,489.8,491.700,512.000
3,P Val,NaN,0.681,0.499


In [161]:
# 1mm/hr Precip Field
Var = "ET Begin 1mm/hr"
ET_Begin_Precip_1mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Precip_1mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,129.0,189.200,161.600
1,Median,290.0,297.500,279.000
2,90%,498.2,494.600,480.800
3,P Val,NaN,0.624,0.339


In [162]:
# 1mm/hr Precip Field
Var = "ET Complete 1mm/hr"
ET_Compl_Precip_1mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Precip_1mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,132.6,145.900,206.400
1,Median,294.0,315.500,315.000
2,90%,469.4,515.500,503.200
3,P Val,NaN,0.767,0.588


In [163]:
# 5mm/hr Precip Field
Var = "Trop Peak 5mm/hr"
Trop_Peak_Precip_5mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Precip_5mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,24.8,21.700,22.800
1,Median,40.0,43.500,48.000
2,90%,74.0,75.600,104.600
3,P Val,NaN,0.263,0.124


In [164]:
# 5mm/hr Precip Field
Var = "ET Begin 5mm/hr"
ET_Begin_Precip_5mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Precip_5mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,21.8,31.700,31.400
1,Median,49.0,57.000,57.000
2,90%,89.2,100.100,109.600
3,P Val,NaN,0.101,0.026


In [165]:
# 5mm/hr Precip Field
Var = "ET Complete 5mm/hr"
ET_Compl_Precip_5mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Precip_5mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,7.0,9.000,22.8
1,Median,40.0,36.000,60.0
2,90%,76.4,85.300,109.0
3,P Val,NaN,0.694,0.0


In [166]:
# 10mm/hr Precip Field
Var = "Trop Peak 10mm/hr"
Trop_Peak_Precip_10mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Precip_10mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,9.8,10.100,14.000
1,Median,20.0,25.000,24.000
2,90%,35.6,46.000,42.600
3,P Val,NaN,0.152,0.123


In [167]:
# 10mm/hr Precip Field
Var = "ET Begin 10mm/hr"
ET_Begin_Precip_10mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Precip_10mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,2.6,7.700,5.800
1,Median,17.0,19.000,22.000
2,90%,35.6,43.900,50.400
3,P Val,NaN,0.175,0.101


In [168]:
# 10mm/hr Precip Field
Var = "ET Complete 10mm/hr"
ET_Compl_Precip_10mmhr_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Precip_10mmhr_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,0.0,0.000,2.0
1,Median,6.0,5.000,15.0
2,90%,18.2,20.300,40.6
3,P Val,NaN,0.259,0.0


In [169]:
# Max Precip Rate
Var = "Trop Peak Max Precip Rate"
Trop_Peak_Precip_Max_Precip_Rate_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Precip_Max_Precip_Rate_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,24.148,22.104,24.342
1,Median,43.255,47.129,50.762
2,90%,56.782,67.840,79.282
3,P Val,NaN,0.074,0.001


In [170]:
# Max Precip Rate
Var = "ET Begin Max Precip Rate"
ET_Begin_Precip_Max_Precip_Rate_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Precip_Max_Precip_Rate_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,11.126,16.859,14.961
1,Median,28.266,30.530,33.888
2,90%,51.797,67.312,75.208
3,P Val,NaN,0.255,0.147


In [171]:
# Max Precip Rate
Var = "ET Complete Max Precip Rate"
ET_Compl_Precip_Max_Precip_Rate_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Precip_Max_Precip_Rate_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,7.238,7.783,11.320
1,Median,13.907,14.280,20.240
2,90%,21.032,24.512,31.627
3,P Val,NaN,0.422,0.000


In [172]:
# Max Precip Water
Var = "Trop Peak Max Precip Water"
Trop_Peak_Precip_Max_Precip_Water_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Precip_Max_Precip_Water_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,71.193,69.739,77.095
1,Median,83.417,93.697,103.970
2,90%,91.442,104.026,117.374
3,P Val,NaN,0.000,0.000


In [173]:
# Max Precip Water
Var = "ET Begin Max Precip Water"
ET_Begin_Precip_Max_Precip_Water_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Precip_Max_Precip_Water_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,55.037,60.044,63.324
1,Median,75.738,80.055,84.421
2,90%,86.312,101.438,111.221
3,P Val,NaN,0.017,0.000


In [174]:
# Max Precip Water
Var = "ET Complete Max Precip Water"
ET_Compl_Precip_Max_Precip_Water_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Precip_Max_Precip_Water_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,44.533,48.415,56.971
1,Median,55.743,59.259,67.197
2,90%,62.156,69.840,75.222
3,P Val,NaN,0.005,0.000


In [175]:
# Total Precip Water
Var = "Trop Peak Total Precip Water"
Trop_Peak_Precip_Total_Precip_Water_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Precip_Total_Precip_Water_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,211977.906,208033.212,228880.656
1,Median,264076.688,286062.172,316680.875
2,90%,296728.769,326221.753,352478.625
3,P Val,NaN,0.000,0.000


In [176]:
# Total Precip Water
Var = "ET Begin Total Precip Water"
ET_Begin_Precip_Total_Precip_Water_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Precip_Total_Precip_Water_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,178514.591,185102.362,198102.469
1,Median,226379.984,252957.383,281366.812
2,90%,265319.494,293750.956,327114.088
3,P Val,NaN,0.000,0.000


In [177]:
# Total Precip Water
Var = "ET Complete Total Precip Water"
ET_Compl_Precip_Total_Precip_Water_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Precip_Total_Precip_Water_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,138307.375,139035.344,167875.609
1,Median,175118.406,195262.367,220670.375
2,90%,211926.334,233562.369,267574.175
3,P Val,NaN,0.005,0.000


In [178]:
# Min Cloud Top Temperature
Var = "Trop Peak Min Cloud Temp"
Trop_Peak_Temp_Cloud_Min_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Temp_Cloud_Min_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-76.468,-73.664,-74.570
1,Median,-67.560,-66.339,-65.416
2,90%,-58.554,-58.565,-56.447
3,P Val,NaN,0.155,0.292


In [179]:
# Min Cloud Top Temperature
Var = "ET Begin Min Cloud Temp"
ET_Begin_Temp_Cloud_Min_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Temp_Cloud_Min_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-74.632,-73.457,-69.872
1,Median,-61.173,-59.729,-57.509
2,90%,-50.618,-50.575,-48.318
3,P Val,NaN,0.755,0.123


In [180]:
# Min Cloud Top Temperature
Var = "ET Complete Min Cloud Temp"
ET_Compl_Temp_Cloud_Min_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Temp_Cloud_Min_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-60.484,-59.654,-59.913
1,Median,-52.225,-50.002,-50.045
2,90%,-45.543,-43.552,-43.855
3,P Val,NaN,0.032,0.020


In [181]:
# Surface Temperature
Var = "Trop Peak Sfc Temp"
Trop_Peak_Temp_Sfc_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Temp_Sfc_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,24.551,26.333,27.540
1,Median,28.357,29.693,30.927
2,90%,29.373,31.442,32.054
3,P Val,NaN,0.000,0.000


In [182]:
# Surface Temperature
Var = "ET Begin Sfc Temp"
ET_Begin_Temp_Sfc_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Temp_Sfc_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,21.609,23.856,23.465
1,Median,27.321,28.025,28.955
2,90%,29.156,30.431,31.684
3,P Val,NaN,0.054,0.000


In [183]:
# Surface Temperature
Var = "ET Complete Sfc Temp"
ET_Compl_Temp_Sfc_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Temp_Sfc_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,17.260,16.924,20.388
1,Median,21.517,23.047,23.893
2,90%,26.640,26.621,27.887
3,P Val,NaN,0.124,0.002


In [184]:
# 850hPa Temperature
Var = "Trop Peak 850hPa Temp"
Trop_Peak_Temp_850hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Temp_850hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,13.806,13.323,15.851
1,Median,16.741,18.159,19.968
2,90%,18.450,19.999,21.519
3,P Val,NaN,0.000,0.000


In [185]:
# 850hPa Temperature
Var = "ET Begin 850hPa Temp"
ET_Begin_Temp_850hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Temp_850hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,11.338,13.402,14.693
1,Median,15.908,18.044,18.810
2,90%,17.743,21.397,21.385
3,P Val,NaN,0.000,0.000


In [186]:
# 850hPa Temperature
Var = "ET Complete 850hPa Temp"
ET_Compl_Temp_850hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Temp_850hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,5.498,7.893,11.069
1,Median,12.803,14.745,16.696
2,90%,16.523,18.973,19.912
3,P Val,NaN,0.014,0.000


In [187]:
# 500hPa Temperature
Var = "Trop Peak 500hPa Temp"
Trop_Peak_Temp_500hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Temp_500hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-9.382,-7.818,-6.978
1,Median,-6.334,-4.305,-2.253
2,90%,-4.331,-2.169,-0.184
3,P Val,NaN,0.000,0.000


In [188]:
# 500hPa Temperature
Var = "ET Begin 500hPa Temp"
ET_Begin_Temp_500hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Temp_500hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-11.204,-8.523,-7.571
1,Median,-6.957,-5.608,-3.651
2,90%,-5.026,-2.869,-0.535
3,P Val,NaN,0.000,0.000


In [189]:
# 500hPa Temperature
Var = "ET Complete 500hPa Temp"
ET_Compl_Temp_500hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Temp_500hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-16.100,-13.200,-9.318
1,Median,-10.405,-7.991,-5.640
2,90%,-6.042,-4.766,-2.935
3,P Val,NaN,0.000,0.000


In [190]:
# 200hPa Temperature
Var = "Trop Peak 200hPa Temp"
Trop_Peak_Temp_200hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Temp_200hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-55.987,-55.757,-53.898
1,Median,-53.656,-50.543,-47.533
2,90%,-50.821,-46.888,-43.503
3,P Val,NaN,0.000,0.000


In [191]:
# 200hPa Temperature
Var = "ET Begin 200hPa Temp"
ET_Begin_Temp_200hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Temp_200hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-57.612,-55.971,-55.653
1,Median,-53.872,-50.598,-47.872
2,90%,-49.487,-46.593,-41.673
3,P Val,NaN,0.000,0.000


In [192]:
# 200hPa Temperature
Var = "ET Complete 200hPa Temp"
ET_Compl_Temp_200hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Temp_200hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,10%,-58.568,-55.972,-55.241
1,Median,-54.546,-52.483,-48.430
2,90%,-49.112,-47.499,-41.783
3,P Val,NaN,0.009,0.000


In [193]:
# 200hPa Zonal Wind
Var = "Trop Peak 200hPa Zonal Wind"
Trop_Peak_U_200hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_U_200hPa_DF

KeyError: 'Trop Peak 200hPa Zonal Wind'

In [ ]:
# 200hPa Zonal Wind
Var = "ET Begin 200hPa Zonal Wind"
ET_Begin_U_200hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_U_200hPa_DF

In [ ]:
# 200hPa Zonal Wind
Var = "ET Complete 200hPa Zonal Wind"
ET_Compl_U_200hPa_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_U_200hPa_DF

In [ ]:
# Max Rising Velocity
Var = "Trop Peak Max Rising"
Trop_Peak_Max_Rising_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Max_Rising_DF

In [ ]:
# Max Rising Velocity
Var = "ET Begin Max Rising"
ET_Begin_Max_Rising_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Max_Rising_DF

In [ ]:
# Max Rising Velocity
Var = "ET Complete Max Rising"
ET_Compl_Max_Rising_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Max_Rising_DF

In [ ]:
# Max Sinking Velocity
Var = "Trop Peak Max Sinking"
Trop_Peak_Max_Sinking_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
Trop_Peak_Max_Sinking_DF

In [ ]:
# Max Sinking Velocity
Var = "ET Begin Max Sinking"
ET_Begin_Max_Sinking_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Begin_Max_Sinking_DF

In [ ]:
# Max Sinking Velocity
Var = "ET Complete Max Sinking"
ET_Compl_Max_Sinking_DF = Percentile_DF(Control_ET_Compo[Var], RCP45_ET_Compo[Var], RCP85_ET_Compo[Var])
ET_Compl_Max_Sinking_DF